In [1]:
import os
import re

import numpy as np
import pandas as pd

In [2]:
# fitting = dfs[0][(dfs[0]["src"] <= seed) & (seed <= dfs[0]["src"] + dfs[0]["rng"])]
# res = fitting["dst"] + seed - fitting["src"]

file_names = sorted(os.listdir("data"))[1:]

seed_column_names = [re.search(r"(.*-){3}(\w+)", name).group(2) for name in file_names]

# Get various mappings
dfs = []
for file in file_names:
    df = pd.read_csv(f"data/{file}", names=("dst", "src", "rng"), sep=" ")
    df.name = re.search(r"^[^ ]*", file).group()
    dfs.append(df)

seed_column_names = ["seed"] + seed_column_names

seed = pd.read_csv("data/1-seeds.csv", sep=" ", header=None, names=seed_column_names)


def find_mapping(src_val, mapping):
    fitting = mapping[
        (mapping["src"] <= src_val) & (src_val < mapping["src"] + mapping["rng"])
    ]
    # print(mapping)
    # print(src_val)
    # print(fitting)
    res = src_val - fitting["src"] + fitting["dst"]
    return src_val if res.empty else res.reset_index(drop=True)[0]

In [3]:
for i in range(len(dfs)):
    seed[seed_column_names[i + 1]] = seed[seed_column_names[i]].apply(
        find_mapping, args=(dfs[i],)
    )
seed

,seed,soil,fertilizer,water,light,temperature,humidity,location
0,104847962,638456097,534117463,651982595,3982774908,3982774908,4283162641,4283162641
1,1212568077,235532501,131193867,887760781,3441510965,1720850774,1936555017,1838498110
2,3890048781,2469591604,2469591604,3787846375,2884874210,3867529169,4167916902,1191329316
3,1520059863,2954179983,3469062691,2906830052,656217636,730688908,657834936,2322923652
4,310308287,843916422,739577788,1009777124,733208466,807679738,2054471866,3191770485
5,3492562455,3947557175,3854335256,1978106847,4255771485,4286815724,2614004927,3300444838
6,1901414562,3335534682,3115051797,1532034748,314636284,417389099,1400031420,3124756470
7,2474299950,1488258798,1481400162,3421966547,1133046171,1890784901,1674770862,3455603946
8,3394639029,3849633749,3756411830,2265712560,1835663583,2253429197,259197691,749391108
9,862612782,437477884,333139250,451004382,3251092485,1773591398,1989295641,1891238734


## Part 2
It's a bit of a hack, but it works well.
I am searching ranges from the lowest location values and inverse them through the mapping to seed number. Then I check if the seed number is within my range. Brute-force, yes, but if the values are randomly spread, it should---and does---work quite nicely.

In [28]:
seeds_ranges = pd.read_csv(
    "data/1-seeds.csv", sep=" ", header=None, names=("low", "rng")
)
seeds_ranges

,low,rng
0,104847962,3583832
1,1212568077,114894281
2,3890048781,333451605
3,1520059863,217361990
4,310308287,12785610
5,3492562455,292968049
6,1901414562,516150861
7,2474299950,152867148
8,3394639029,59690410
9,862612782,176128197


In [14]:
def find_mapping_vec(src_val, mapping):
    mapping_vect = mapping.values
    index = np.where(
        (mapping_vect[:, 1] <= src_val)
        & (src_val < mapping_vect[:, 1] + mapping_vect[:, 2])
    )
    fitted_vec = mapping_vect[index]
    if fitted_vec.shape[0] == 0:
        return src_val
    res = src_val - fitted_vec[0][1] + fitted_vec[0][0]
    return res


def find_mapping_inverse_vec(dst_val, mapping):
    mapping_vect = mapping.values
    index = np.where(
        (mapping_vect[:, 0] <= dst_val)
        & (dst_val < mapping_vect[:, 0] + mapping_vect[:, 2])
    )
    fitted_vec = mapping_vect[index]
    if fitted_vec.shape[0] == 0:
        return dst_val
    res = dst_val - fitted_vec[0][0] + fitted_vec[0][1]
    return res

In [29]:
seeds_ranges["max"] = seeds_ranges["low"] + seeds_ranges["rng"]

In [62]:
foundit = False
for j in range(10, 50):
    print(f"Running range {100000*j}, {100000*(j+1)}")
    locat = pd.DataFrame(range(100000 * j, 100000 * (j + 1)), columns=("location",))
    locat

    locat_column_names = seed_column_names[::-1]
    for i in range(len(dfs)):
        locat[locat_column_names[i + 1]] = locat[locat_column_names[i]].apply(
            find_mapping_inverse_vec, args=(dfs[-(i + 1)],)
        )

    seeds_ranges_vec = seeds_ranges.values

    for seed in locat["seed"]:
        if (
            np.where(
                (seeds_ranges_vec[:, 0] <= seed) & (seed < seeds_ranges_vec[:, 2])
            )[0].size
            != 0
        ):
            print(seed)
            foundit = True
            break

    if foundit:
        break

Running range 1000000, 1100000
Running range 1100000, 1200000
Running range 1200000, 1300000
Running range 1300000, 1400000
Running range 1400000, 1500000
Running range 1500000, 1600000
Running range 1600000, 1700000
Running range 1700000, 1800000
Running range 1800000, 1900000
Running range 1900000, 2000000
Running range 2000000, 2100000
2243422640


In [64]:
part2_seed = pd.DataFrame([2243422640], columns=("seed",))
                          
for i in range(len(dfs)):
    part2_seed[seed_column_names[i + 1]] = part2_seed[seed_column_names[i]].apply(
        find_mapping, args=(dfs[i],)
    )
part2_seed

,seed,soil,fertilizer,water,light,temperature,humidity,location
0,2243422640,2222566796,1421598213,3362164598,1073244222,2942497292,2986998165,2008785


## Attemp for full table of seeds (unsuccesful)

In [17]:
# src_val = 533608135
# dst_val = 1067216283
# df_vect = dfs[0]
# mapping_vect = df_vect.values
# index = np.where(
#     (mapping_vect[1] <= src_val) & (src_val < mapping_vect[1] + mapping_vect[2])
# )
# fitted_vec = mapping_vect[index][0]
# fitted_vec
# res = src_val - fitted_vec[1] + fitted_vec[0]
# res

# find_mapping(src_val, dfs[0])
# #find_mapping_inverse_vec(dst_val, dfs[0])
# #dfs[0]

IndexError: index 2 is out of bounds for axis 0 with size 2

In [ ]:
seed = pd.DataFrame(
    range(0, 2),
    columns=(seed_column_names[-1],),
)
seed

In [ ]:
seed[seed_column_names[-1]].apply(find_mapping_inverse_vec, args=(dfs[-1],))

In [ ]:
min_values = []


print(f"running series {1}")
for j in range(len(dfs)):
    # print(seed_column_names[-(j + 2)])
    # print(seed_column_names[-(j + 1)])
    # print(dfs[-(j+1)].name)
    seed[seed_column_names[-(j + 2)]] = seed[seed_column_names[-(j + 1)]].apply(
        find_mapping_inverse_vec, args=(dfs[-(j + 1)],)
    )
    print(f"finished dfs mapping {len(dfs)-j}")

seed

In [ ]:
seed

In [ ]:
seed[seed_column_names[0]].apply(
    find_mapping_vec,
    args=(dfs[0],),
)

In [ ]:
min_values = []

for i in range(len(seed_ranges)):
    seed = pd.DataFrame(
        range(seed_ranges.iloc[i, 0], seed_ranges.iloc[i, 0] + seed_ranges.iloc[i, 1]),
        columns=(seed_column_names[0],),
    )
    print(f"running series {i}")
    for j in range(len(dfs)):
        seed[seed_column_names[j + 1]] = seed[seed_column_names[j]].apply(
            find_mapping, args=(dfs[j],)
        )
        print(f"finished dfs mapping {j}")

    min_values.append(seed["location"].min())
    del seed

In [ ]:
min_values